<a href="https://colab.research.google.com/github/Ishraqalibon/fine-tune-gemma-2-2B/blob/main/fine_tune_gemma_2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q -U trl==0.8.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.2 MB/s eta 0:00:00


In [11]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.2
    Uninstalling bitsandbytes-0.43.2:
      Successfully uninstalled bitsandbytes-0.43.2


In [1]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [2]:
os.environ['HF_TOKEN']=userdata.get('HF_TOKEN')

In [3]:
model_id="google/gemma-2-2b"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)

In [4]:
tokenizer=AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={'':0},
    token=os.environ['HF_TOKEN']
)

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [6]:
text = "Quote: Imagination is more,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate( ** inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Quote: Imagination is more, more, more than the intellect can ever hope to be.

-Albert Einstein

I have always


In [7]:
os.environ['WANDB_DISABLED']='false'

In [8]:
lora_config = LoraConfig(
    r =8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj","gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [9]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [10]:
def formatting_func(example):
  text = f"IQuote: {example['quote' ][0]}\nAuthor: {example['author' ][0]}"
  return [text]

In [11]:
data['train']

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

In [22]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      warmup_steps=2,
      max_steps=100,
      learning_rate=2e-4,
      fp16=True,
      logging_steps=1,
      output_dir="outputs",
      optim="paged_adamw_8bit"
    ),

    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `t

In [23]:
trainer.train()

Step,Training Loss
1,2.876300
2,1.870400
3,2.475200
4,2.716200
5,1.910400
6,2.182200
7,2.855800
8,1.868500
9,3.140300
10,2.077300


TrainOutput(global_step=100, training_loss=1.9384833127260208, metrics={'train_runtime': 203.5563, 'train_samples_per_second': 1.965, 'train_steps_per_second': 0.491, 'total_flos': 193860972094464.0, 'train_loss': 1.9384833127260208, 'epoch': 0.1594896331738437})

In [29]:
text = "Quote: You've gotta dance like there's nobody "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate( ** inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Quote: You've gotta dance like there's nobody 
watching, love like you'll never be hurt, 
sing like there's nobody listening, 
and live like it's heaven on earth.

- William W. Purkey

I'm not sure if I'


In [30]:
text = "Quote: You've gotta dance like there's nobody "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate( ** inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: You've gotta dance like there's nobody 
watching, love like you'll never be hurt, 
sing like there's nobody listening, 
and live like it's heaven on earth.

- William W. Pur
